#Instruções de Uso:
1. Importe os arquivos do modelo de regressão logística (`modelo_regressao_logistica.pkl`) e do vetorizador (`tfidf_vectorizer.pkl`) para a seção de arquivos deste notebook.
2. Importe o arquivo contendo as avaliações dos clientes.
3. Na barra de ferramentas, clique na opção "Ambiente de execução->Executar tudo".
4. Baixe o arquivo "analise_sentimento" da seção de arquivos.

obs: A coluna contendo as avaliações que serão processadas precisa chamar "avaliacao".

In [14]:
import pandas as pd

In [31]:
df = pd.read_csv('dataset_novas_avaliacoes.csv')
df_avaliacoes = df['avaliacao'].copy()

In [30]:
#instalando a biblioteca nltk
import nltk

nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [9]:
! pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.4 MB/s eta 0:00:00


In [10]:
import unidecode

In [2]:
import joblib
tfidf = joblib.load('tfidf_vectorizer.pkl')
regressao_logistica = joblib.load('modelo_regressao_logistica.pkl')

In [8]:
from nltk import tokenize

palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')
token_pontuacao = tokenize.WordPunctTokenizer()
stemmer = nltk.RSLPStemmer()


def processar_avaliacao(avaliacao):
  # passo 1
  tokens = token_pontuacao.tokenize(avaliacao)

  # passo 2
  frase_processada = [palavra for palavra in tokens if palavra.lower() not in palavras_irrelevantes]

  # passo 3
  frase_processada = [palavra for palavra in frase_processada if palavra.isalpha()]

  # passo 4
  frase_processada = [unidecode.unidecode(palavra) for palavra in frase_processada]

  # passo 5
  frase_processada = [stemmer.stem(palavra) for palavra in frase_processada]

  return ' '.join(frase_processada)

In [32]:
novas_avaliacoes_processadas = [processar_avaliacao(avaliacao) for avaliacao in df_avaliacoes]

In [33]:
novas_avaliacoes_tfidf = tfidf.transform(novas_avaliacoes_processadas)

predicoes = regressao_logistica.predict(novas_avaliacoes_tfidf)

df_previsoes = pd.DataFrame({
    'Avaliação': df_avaliacoes,
    'Sentimento previsto': predicoes
})

df_previsoes

,Avaliação,Sentimento previsto
0,"Esse smartphone superou minhas expectativas, r...",positivo
1,o cheiro e muito desagradavel!! pra quem se in...,positivo
2,"Eu fiz a compra a loja não entregou o produto,...",negativo
3,"Atendeu minhas expectativas, só achei a luz ru...",negativo
4,"Gostei muito, só achei que ficou a desejar a q...",positivo
...,...,...
15496,Bommm.mmmmmmmmmmmmmmmm Mmmmmmmmmmmmmmmmmmmmmmm...,positivo
15497,"O produto seca o cabelo, mas tem que passar di...",negativo
15498,"Muito simples de mexer, o manual explica direi...",negativo
15499,"Produto muito bom, exatamente como na internet...",positivo


In [35]:
df_previsoes.to_excel('analise_sentimento.xlsx', index=False)